<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Using_numba_cuda_max_independent_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-12.5/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-12.5/nvvm/lib64/libnvvm.so"

In [12]:
!uv pip install -q --system numba-cuda==0.15

In [13]:
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [14]:
%%writefile k16.txt
16
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k16.txt


In [15]:
%%writefile k18.txt
18
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k18.txt


In [16]:
%%writefile k20.txt
20
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k20.txt


In [17]:
%%writefile python_cuda.py
import sys
import time
import numpy as np
from numba import cuda

#import cupy as cp

# A reduction on the GPU
@cuda.reduce
def max_reduce(a, b):
    if a < b:
        return b
    else:
        return a

def read_adjacency_matrix(file_name):
  file_object = open(file_name, "r")
  # Input the number of rows and columns
  size = int(file_object.readline())
  rows = size
  cols = size
  # Initialize an empty matrix
  matrix = []

  # Input the matrix elements
  for i in range(rows):
    row = list(map(int, file_object.readline().split()))
    matrix.append(row)
  return matrix,size

@cuda.jit
def convert_from_int_to_set(set_of_nodes,integer,size):
  mask = 1
  for i in range(size):
    if ((mask & integer) != 0):
      set_of_nodes[i] = 1
    else:
      set_of_nodes[i] = 0
    mask = mask * 2


@cuda.jit
def kernel_find_max_ind_set(adj_mat,array_with_sizes,size,n_nodes):
  i = cuda.grid(1)
  #i = 3
  if (i >= size):
    return
  set_of_nodes = cuda.local.array(32,dtype=np.float64)

  convert_from_int_to_set(set_of_nodes,i,n_nodes)
  is_independent = True

  for n1 in range(n_nodes-1):
    if (set_of_nodes[n1] == 1):
      for n2 in range(n1+1,n_nodes):
          if (set_of_nodes[n2] == 1):
            if (adj_mat[n1*n_nodes+n2] == 1):
              is_independent = False
              break


  if (is_independent==True):
    countBits = 0
    mask = 1
    for index in range(n_nodes):
      if ((mask & i) != 0):
        countBits += 1
      mask = mask * 2
    array_with_sizes[i] = countBits
  else:
    array_with_sizes[i] = 0


# Find the maximum independent set
def find_max_ind_set(adj_matrix,size):

  max_independent_set_size = 0


  flat_adj_matrix = np.reshape(adj_matrix,size*size)

  adj_mat_cuda = cuda.to_device(flat_adj_matrix)
  size_of_power_set = 1
  for i in range(size):
    size_of_power_set *= 2
  # print("The power set has ",size_of_power_set," elements")

  array_with_sizes = np.zeros(size_of_power_set,dtype=np.int32)
  array_with_sizes_device = cuda.to_device(array_with_sizes)
  cuda.synchronize()

  threads_per_block = 256
  blocks_per_grid = (size_of_power_set + threads_per_block - 1) // threads_per_block
  # print(blocks_per_grid,threads_per_block)
  kernel_find_max_ind_set[blocks_per_grid, threads_per_block](adj_mat_cuda,array_with_sizes_device,size_of_power_set,size)


  cuda.synchronize()
  array_with_sizes_device.copy_to_host(array_with_sizes)

  max_independent_set_size = np.max(array_with_sizes)
  return max_independent_set_size



if __name__ == "__main__":
# Read the content of the file passed in the command line
# that contain the adjacency matrix of the graph
# whose maximum independent set we are calculating

  adj_matrix,size = read_adjacency_matrix(sys.argv[1])

  start_time = time.time()
  max_independent_set_size = find_max_ind_set(adj_matrix,size)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Time required to carry out the computation: ",elapsed_time)
  print("The size of the maximum independent set is: ",max_independent_set_size)

Overwriting python_cuda.py


In [18]:
!python python_cuda.py k16.txt

Time required to carry out the computation:  0.5322046279907227
The size of the maximum independent set is:  1


In [19]:
!python python_cuda.py k18.txt

Time required to carry out the computation:  0.5314874649047852
The size of the maximum independent set is:  1


In [20]:
!python python_cuda.py k20.txt

Time required to carry out the computation:  0.513791561126709
The size of the maximum independent set is:  1


A different test file where four nodes are completely disconnected from each other. The maximum independent set for this graph is of size 4. The disconnected nodes are nodes 0 through 3.

In [23]:
%%writefile test16.txt
16
0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting test16.txt


In [24]:
!python python_cuda.py test16.txt

Time required to carry out the computation:  0.5810809135437012
The size of the maximum independent set is:  4


Another  test file where six nodes are completely disconnected from each other. The maximum independent set for this graph is of size 6. The disconnected nodes are nodes 0 through 5.

In [25]:
%%writefile test16b.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing test16b.txt


In [26]:
!python python_cuda.py test16b.txt

Time required to carry out the computation:  0.5642678737640381
The size of the maximum independent set is:  6


In [27]:
%%writefile test16c.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0

Writing test16c.txt


In [28]:
!python python_cuda.py test16c.txt

Time required to carry out the computation:  0.5473628044128418
The size of the maximum independent set is:  6
